<a href="https://colab.research.google.com/github/SalarShafiee/ML-Projects/blob/main/Masterarbeit_GISMA_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import os
import gc
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm


AUTOTUNE = tf.data.experimental.AUTOTUNE

In [3]:
import kagglehub

path = kagglehub.dataset_download("zygmuntyt/nutrition5k-dataset-side-angle-images")

print("Path to dataset files:", path)

100%|██████████| 5.97G/5.97G [05:02<00:00, 21.2MB/s]


Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/zygmuntyt/nutrition5k-dataset-side-angle-images/versions/2


In [2]:
import pandas as pd
df=pd.read_excel('/content/sample_data/dish_metadata_cafe1 .xlsx')
df.head()

,dish_id,total_calories,total_mass,total_fat,total_carb,total_protein
0,dish_1550704750,136.949997,83.0,2.988,0.00,25.73
1,dish_1550704903,30.400000,76.0,0.152,6.84,1.52
2,dish_1550705288,77.000000,100.0,0.100,17.00,2.00
3,dish_1550705370,122.500000,49.0,7.350,0.00,12.74
4,dish_1550705445,4.000000,40.0,0.000,0.80,0.40


In [3]:
import pandas as pd
import os
from glob import glob

# Assuming df is already loaded
df['path_to_images'] = ''

base_path = "/root/.cache/kagglehub/datasets/zygmuntyt/nutrition5k-dataset-side-angle-images/versions/2"

for index, row in df.iterrows():
    dish_id = row['dish_id']
    image_folder = os.path.join(base_path, dish_id, "frames_sampled30")

    if os.path.exists(image_folder):
        # Collect all image paths within the folder
        image_paths = glob(os.path.join(image_folder, "*.jpeg"))
        if image_paths:
            df.at[index, 'path_to_images'] = image_paths  # Store list of paths
        else:
            print(f"Warning: No images found for dish_id: {dish_id}")
            df.at[index, 'path_to_images'] = []
    else:
        print(f"Warning: Folder not found for dish_id: {dish_id}")
        df.at[index, 'path_to_images'] = []

df.head()

,dish_id,total_calories,total_mass,total_fat,total_carb,total_protein,path_to_images
0,dish_1550704750,136.949997,83.0,2.988,0.00,25.73,[/root/.cache/kagglehub/datasets/zygmuntyt/nut...
1,dish_1550704903,30.400000,76.0,0.152,6.84,1.52,[/root/.cache/kagglehub/datasets/zygmuntyt/nut...
2,dish_1550705288,77.000000,100.0,0.100,17.00,2.00,[/root/.cache/kagglehub/datasets/zygmuntyt/nut...
3,dish_1550705370,122.500000,49.0,7.350,0.00,12.74,[/root/.cache/kagglehub/datasets/zygmuntyt/nut...
4,dish_1550705445,4.000000,40.0,0.000,0.80,0.40,[/root/.cache/kagglehub/datasets/zygmuntyt/nut...


In [4]:


import pandas as pd

# Assuming 'path_to_images' is the correct column
df.loc[df['path_to_images'].apply(lambda x: x == [] or x == ['image not found']), 'path_to_images'] = pd.NA

df

,dish_id,total_calories,total_mass,total_fat,total_carb,total_protein,path_to_images
0,dish_1550704750,136.949997,83.0,2.988000,0.000000,25.730000,[/root/.cache/kagglehub/datasets/zygmuntyt/nut...
1,dish_1550704903,30.400000,76.0,0.152000,6.840000,1.520000,[/root/.cache/kagglehub/datasets/zygmuntyt/nut...
2,dish_1550705288,77.000000,100.0,0.100000,17.000000,2.000000,[/root/.cache/kagglehub/datasets/zygmuntyt/nut...
3,dish_1550705370,122.500000,49.0,7.350000,0.000000,12.740000,[/root/.cache/kagglehub/datasets/zygmuntyt/nut...
4,dish_1550705445,4.000000,40.0,0.000000,0.800000,0.400000,[/root/.cache/kagglehub/datasets/zygmuntyt/nut...
...,...,...,...,...,...,...,...
4763,dish_1568666230,55.800003,268.0,0.692000,10.969999,4.532000,[/root/.cache/kagglehub/datasets/zygmuntyt/nut...
4764,dish_1568666263,93.400002,348.0,0.852000,19.770000,7.172000,[/root/.cache/kagglehub/datasets/zygmuntyt/nut...
4765,dish_1568666303,443.199982,560.0,8.483999,19.770000,72.892006,[/root/.cache/kagglehub/datasets/zygmuntyt/nut...
4766,dish_1568666329,743.760010,594.0,42.484001,19.770000,72.891998,[/root/.cache/kagglehub/datasets/zygmuntyt/nut...


In [5]:

df = df.dropna()
df

,dish_id,total_calories,total_mass,total_fat,total_carb,total_protein,path_to_images
0,dish_1550704750,136.949997,83.0,2.988000,0.000000,25.730000,[/root/.cache/kagglehub/datasets/zygmuntyt/nut...
1,dish_1550704903,30.400000,76.0,0.152000,6.840000,1.520000,[/root/.cache/kagglehub/datasets/zygmuntyt/nut...
2,dish_1550705288,77.000000,100.0,0.100000,17.000000,2.000000,[/root/.cache/kagglehub/datasets/zygmuntyt/nut...
3,dish_1550705370,122.500000,49.0,7.350000,0.000000,12.740000,[/root/.cache/kagglehub/datasets/zygmuntyt/nut...
4,dish_1550705445,4.000000,40.0,0.000000,0.800000,0.400000,[/root/.cache/kagglehub/datasets/zygmuntyt/nut...
...,...,...,...,...,...,...,...
4763,dish_1568666230,55.800003,268.0,0.692000,10.969999,4.532000,[/root/.cache/kagglehub/datasets/zygmuntyt/nut...
4764,dish_1568666263,93.400002,348.0,0.852000,19.770000,7.172000,[/root/.cache/kagglehub/datasets/zygmuntyt/nut...
4765,dish_1568666303,443.199982,560.0,8.483999,19.770000,72.892006,[/root/.cache/kagglehub/datasets/zygmuntyt/nut...
4766,dish_1568666329,743.760010,594.0,42.484001,19.770000,72.891998,[/root/.cache/kagglehub/datasets/zygmuntyt/nut...


In [6]:
df.shape

(4606, 7)

In [7]:
import pandas as pd
import ast
import tensorflow as tf

# Instead of ast.literal_eval, directly access the first element of the list
def get_first_image(path_list):
    # Check if the path_list is valid and not empty
    if isinstance(path_list, list) and path_list:
        return path_list[0]  # Return the first image path
    else:
        return None  # or handle the case appropriately (e.g., return an empty string)

# Create a new column with the first image path
df['first_image'] = df['path_to_images'].apply(get_first_image)

# For debugging, you might want to check the first few rows:
print(df[['first_image', 'total_carb']].head())

                                         first_image  total_carb
0  /root/.cache/kagglehub/datasets/zygmuntyt/nutr...        0.00
1  /root/.cache/kagglehub/datasets/zygmuntyt/nutr...        6.84
2  /root/.cache/kagglehub/datasets/zygmuntyt/nutr...       17.00
3  /root/.cache/kagglehub/datasets/zygmuntyt/nutr...        0.00
4  /root/.cache/kagglehub/datasets/zygmuntyt/nutr...        0.80


<ipython-input-7-270a62e65021>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['first_image'] = df['path_to_images'].apply(get_first_image)


In [8]:
import pandas as pd
import ast
import tensorflow as tf
from sklearn.model_selection import train_test_split

# -------------------------------
# 2. Split Data: Train, Validation, Test
# -------------------------------

# For example, 70% training, 15% validation, 15% test
train_val_df, test_df = train_test_split(df, test_size=0.15, random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.1765, random_state=42)
# Note: 0.1765 ≈ 15% of original data (15/85)

print(f"Train samples: {len(train_df)}, Val samples: {len(val_df)}, Test samples: {len(test_df)}")

# -------------------------------
# 3. Define Data Augmentation and Preprocessing
# -------------------------------

IMG_SIZE = 128
BATCH_SIZE = 16  # Use 16 here; if memory is limited, consider reducing this value.

# Define a data augmentation pipeline using Keras preprocessing layers.
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])

def load_and_preprocess_image(image_path, target, augment=False):
    # Read the image file
    image = tf.io.read_file(image_path)
    # Decode the image. Assuming JPEG; use decode_png for PNG images.
    image = tf.image.decode_jpeg(image, channels=3)
    # Resize the image to the desired size
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    # Normalize pixel values to [0, 1]
    image = image / 255.0
    # Apply data augmentation if requested
    if augment:
        image = data_augmentation(image)
    # Ensure the target is a float32 tensor
    return image, tf.cast(target, tf.float32)

# Helper function to create a dataset from a dataframe.
def create_dataset(df, augment=False, shuffle=False):
    paths = df['first_image'].values
    targets = df['total_carb'].values
    dataset = tf.data.Dataset.from_tensor_slices((paths, targets))
    # Map the load_and_preprocess_image function
    dataset = dataset.map(lambda path, target: load_and_preprocess_image(path, target, augment=augment),
                          num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return dataset

# Create datasets for training, validation, and testing.
train_ds = create_dataset(train_df, augment=False, shuffle=False).cache().prefetch(tf.data.AUTOTUNE)
val_ds = create_dataset(val_df, augment=False, shuffle=False).cache().prefetch(tf.data.AUTOTUNE)
test_ds = create_dataset(test_df, augment=False, shuffle=False).cache().prefetch(tf.data.AUTOTUNE)

Train samples: 3224, Val samples: 691, Test samples: 691


In [14]:
! pip install vit-keras

ERROR: Operation cancelled by user


In [3]:
!pip install tensorflow-addons

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 28.3 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.1
    Uninstalling typeguard-4.4.1:
      Successfully uninstalled typeguard-4.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.5.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [15]:
! pip install tensorflow==2.15.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.2
    Uninstalling wrapt-1.17.2:
      Successfully uninstalled wrapt-1.17.2
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: keras
    Found existing installation: keras 3.8.0
    Uninstalling keras-3.8.0:
      Successfully uninstalled keras-3.8.0
  Attempting uninstall: tensorboard
    Found existing installation

In [9]:
# -------------------------------
# 4. Build the Model Using a Pre-trained ViT
# -------------------------------
# Using vit-keras for the Vision Transformer (ViT) model.
from tensorflow import keras
from vit_keras import vit  # Ensure you have installed vit-keras (pip install vit-keras)

# Load the pre-trained ViT model without its top classification layer.
vit_model = vit.vit_b16(
    image_size=IMG_SIZE,
    pretrained=True,
    include_top=False,      # Exclude the default classification head
    pretrained_top=False    # Do not load the default top weights
)

# Build a new Keras model for regression.
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = vit_model(inputs)  # Extract features from the ViT
# Remove the GlobalAveragePooling2D layer as it's likely already included in the ViT model.
# x = keras.layers.GlobalAveragePooling2D()(x)  # Global average pooling
outputs = keras.layers.Dense(1)(x)  # Regression output: one value for total_carb
model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()

/usr/local/lib/python3.11/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/usr/local/lib/python3.11/dist-packages/vit_keras/utils.py:81: UserWarning: Resizing position embeddings from 24, 24 to 8, 8
  warnings.warn(


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 vit-b16 (Functional)        (None, 768)               85697280  
                                                                 
 dense (Dense)               (None, 1)                 769       
                                                                 
Total params: 85698049 (326.91 MB)
Trainable params: 85698049 (326.91 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


In [ ]:
# -------------------------------
# 5. Compile and Train the Model
# -------------------------------

# Compile the model with an Adam optimizer and Mean Squared Error loss for regression.
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss='mse',
    metrics=['mae']  # Using Mean Absolute Error as an additional metric
)

NUM_EPOCHS = 10  # Set the number of epochs as needed

history = model.fit(
    train_ds,
    epochs=NUM_EPOCHS,
    validation_data=val_ds
)



Epoch 1/10
202/202 [==============================] - 2092s 10s/step - loss: 863.8975 - mae: 11.1951 - val_loss: 947.3446 - val_mae: 9.9915
Epoch 2/10
202/202 [==============================] - 2013s 10s/step - loss: 819.4475 - mae: 10.0631 - val_loss: 903.2003 - val_mae: 9.2932
Epoch 3/10
202/202 [==============================] - 2014s 10s/step - loss: 778.0500 - mae: 8.8728 - val_loss: 863.9421 - val_mae: 8.3642
Epoch 4/10
202/202 [==============================] - 2018s 10s/step - loss: 752.1958 - mae: 8.1671 - val_loss: 850.8108 - val_mae: 8.6567
Epoch 5/10
202/202 [==============================] - 1990s 10s/step - loss: 699.1579 - mae: 6.7233 - val_loss: 775.7127 - val_mae: 7.2092
Epoch 6/10
202/202 [==============================] - 2000s 10s/step - loss: 667.2076 - mae: 5.4645 - val_loss: 765.1640 - val_mae: 6.7278
Epoch 7/10
 20/202 [=>............................] - ETA: 27:55 - loss: 1509.1636 - mae: 7.1321

In [ ]:
# Evaluate on the test set
test_loss, test_mae = model.evaluate(test_ds)
print(f"Test Loss: {test_loss:.4f}, Test MAE: {test_mae:.4f}")